In [105]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import scipy as sp
import statsmodels.api as sm
import statsmodels.formula.api as smf
import sklearn
from sklearn.linear_model import LogisticRegression, Lasso
from sklearn.model_selection import GridSearchCV
os.chdir('C:\\Users\\Kim\\Desktop\\빅콘테스트')

In [106]:
print('python', sys.version)
print('pandas', pd.__version__)
print('numpy', np.__version__)
print('matplotlib', matplotlib.__version__)
print('seaborn', sns.__version__)
print('scipy', sp.__version__)
print('statsmodels', sm.__version__)
print('sklearn', sklearn.__version__)

python 3.7.4 (default, Aug  9 2019, 18:34:13) [MSC v.1915 64 bit (AMD64)]
pandas 0.25.1
numpy 1.16.5
matplotlib 3.2.1
seaborn 0.9.0
scipy 1.4.1
statsmodels 0.10.1
sklearn 0.23.2


# 투수 모델링(재현)

In [107]:
#데이터 불러오기
YEAR_LIST = [2016, 2017, 2018, 2019]
YEAR_LIST2 = [2016, 2017, 2018, 2019, 2020]
for i in YEAR_LIST:
    s1 = "pitcher_%d = pd.read_csv(r'C:\\Users\\Kim\\Desktop\\빅콘테스트\\데이터\\기록\\개인투수\\2020빅콘테스트_스포츠투아이_제공데이터_개인투수_%d.csv', encoding='cp949')"%(i, i)
    exec(s1)
    
for i in YEAR_LIST2:
    s2 = "player_%d = pd.read_csv(r'C:\\Users\\Kim\\Desktop\\빅콘테스트\\데이터\\기록\\선수\\2020빅콘테스트_스포츠투아이_제공데이터_선수_%d.csv', encoding='cp949')"%(i, i)
    exec(s2)

#크롤링 데이터    
pitcher_2020 = pd.read_csv('2020최신투수(선수별).csv')

In [108]:
#선수이름 데이터 생성
for i in YEAR_LIST2:
    s1 = "player_%d = pd.concat([player_%d['PCODE'], player_%d['NAME'], player_%d['AGE_VA']], axis = 1).rename(columns = {'PCODE' : 'P_ID'})"%(i, i, i, i)
    exec(s1)
    
#기록데이터, 선수이름 데이터 병합    
for i in YEAR_LIST:
    s1 = "pitcher_%d = pd.merge(pitcher_%d, player_%d, on ='P_ID', how = 'left')"%(i, i, i)
    exec(s1)


#사용하는 변수만 연도별로 뽑아놓기
for i in YEAR_LIST2:
    s1 = "pitcher_%d = pitcher_%d[['GDAY_DS', 'NAME', 'T_ID','ER', 'INN2', 'BF', 'PA', 'AB', 'HIT', 'H2','H3', 'HR', 'BB', 'IB', 'HP', 'KK', 'R']]"%(i,i)
    exec(s1)
      
#INN2 0제거
for i in YEAR_LIST2:
    s1 = "drop_in_%d = pitcher_%d.loc[pitcher_%d['INN2'] == 0].index"%(i, i, i)
    exec(s1)
    
for i in YEAR_LIST:
    s2 = "pitcher_%d = pitcher_%d.drop(drop_in_%d)"%(i, i, i)
    exec(s2)

In [109]:
#잔여경기 일정에 맞게 날짜 데이터 추출

#기어 잔여경기 29경기
TEAM_LIST = ['HT']
for i in YEAR_LIST2:
    for j in TEAM_LIST:
        s1 = "%s_%d = pitcher_%d.loc[pitcher_%d['T_ID'] == j]['GDAY_DS'].drop_duplicates().reset_index(drop = True)[:115]"%(j, i, i, i)
        exec(s1)
        
#NC, 롯데 잔여경기 28경기
TEAM_LIST = ['NC', 'LT']
for i in YEAR_LIST2:
    for j in TEAM_LIST:
        s1 = "%s_%d = pitcher_%d.loc[pitcher_%d['T_ID'] == j]['GDAY_DS'].drop_duplicates().reset_index(drop = True)[:116]"%(j, i, i, i)
        exec(s1)
        
#KT 잔여경기 27경기        
TEAM_LIST = ['KT']
for i in YEAR_LIST2:
    for j in TEAM_LIST:
        s1 = "%s_%d = pitcher_%d.loc[pitcher_%d['T_ID'] == j]['GDAY_DS'].drop_duplicates().reset_index(drop = True)[:117]"%(j, i, i, i)
        exec(s1)
        
#한화, 삼성 잔여경기 26경기        
TEAM_LIST = ['HH', 'SS']
for i in YEAR_LIST2:
    for j in TEAM_LIST:
        s1 = "%s_%d = pitcher_%d.loc[pitcher_%d['T_ID'] == j]['GDAY_DS'].drop_duplicates().reset_index(drop = True)[:118]"%(j, i, i, i)
        exec(s1)
        
#두산, LG 잔여경기 25경기        
TEAM_LIST = ['OB', 'LG']
for i in YEAR_LIST2:
    for j in TEAM_LIST:
        s1 = "%s_%d = pitcher_%d.loc[pitcher_%d['T_ID'] == j]['GDAY_DS'].drop_duplicates().reset_index(drop = True)[:119]"%(j, i, i, i)
        exec(s1)
        
#SK 잔여경기 24경기        
TEAM_LIST = ['SK']
for i in YEAR_LIST2:
    for j in TEAM_LIST:
        s1 = "%s_%d = pitcher_%d.loc[pitcher_%d['T_ID'] == j]['GDAY_DS'].drop_duplicates().reset_index(drop = True)[:120]"%(j, i, i, i)
        exec(s1)
        
#키움 잔여경기 20경기        
TEAM_LIST = ['WO']
for i in YEAR_LIST2:
    for j in TEAM_LIST:
        s1 = "%s_%d = pitcher_%d.loc[pitcher_%d['T_ID'] == j]['GDAY_DS'].drop_duplicates().reset_index(drop = True)[:124]"%(j, i, i, i)
        exec(s1)

In [110]:
#잔여경기를 제외한 경기 기록(Train)
TEAM_LIST = ['LG', 'SS', 'HH', 'LT', 'NC', 'HT', 'WO', 'SK', 'KT', 'OB']
for i in YEAR_LIST2:
    for j in TEAM_LIST:
        s2 = "train_%s_%d = pd.merge(%s_%d, pitcher_%d.loc[pitcher_%d['T_ID'] == j], on = 'GDAY_DS', how = 'left')"%(j, i, j, i, i, i)
        exec(s2)

In [111]:
#잔여경기를 제외한 경기 기록(Train)
TEAM_LIST = ['LG', 'SS', 'HH', 'LT', 'NC', 'HT', 'WO', 'SK', 'KT', 'OB']
for i in YEAR_LIST2:
    for j in TEAM_LIST:
        s2 = "train_%s_%d = pd.merge(%s_%d, pitcher_%d.loc[pitcher_%d['T_ID'] == j], on = 'GDAY_DS', how = 'left')"%(j, i, j, i, i, i)
        exec(s2)
        
#기어 잔여경기 29경기
TEAM_LIST = ['HT']
for i in YEAR_LIST2:
    for j in TEAM_LIST:
        s1 = "%s_%d_ER = pitcher_%d.loc[pitcher_%d['T_ID'] == j]['GDAY_DS'].drop_duplicates().reset_index(drop = True)[115:]"%(j, i, i, i)
        exec(s1)
        
#NC, 롯데 잔여경기 28경기
TEAM_LIST = ['NC', 'LT']
for i in YEAR_LIST2:
    for j in TEAM_LIST:
        s1 = "%s_%d_ER = pitcher_%d.loc[pitcher_%d['T_ID'] == j]['GDAY_DS'].drop_duplicates().reset_index(drop = True)[116:]"%(j, i, i, i)
        exec(s1)
        
#KT 잔여경기 27경기        
TEAM_LIST = ['KT']
for i in YEAR_LIST2:
    for j in TEAM_LIST:
        s1 = "%s_%d_ER = pitcher_%d.loc[pitcher_%d['T_ID'] == j]['GDAY_DS'].drop_duplicates().reset_index(drop = True)[117:]"%(j, i, i, i)
        exec(s1)
        
#한화, 삼성 잔여경기 26경기        
TEAM_LIST = ['HH', 'SS']
for i in YEAR_LIST2:
    for j in TEAM_LIST:
        s1 = "%s_%d_ER = pitcher_%d.loc[pitcher_%d['T_ID'] == j]['GDAY_DS'].drop_duplicates().reset_index(drop = True)[118:]"%(j, i, i, i)
        exec(s1)
        
#두산, LG 잔여경기 25경기        
TEAM_LIST = ['OB', 'LG']
for i in YEAR_LIST2:
    for j in TEAM_LIST:
        s1 = "%s_%d_ER = pitcher_%d.loc[pitcher_%d['T_ID'] == j]['GDAY_DS'].drop_duplicates().reset_index(drop = True)[119:]"%(j, i, i, i)
        exec(s1)
        
#SK 잔여경기 24경기        
TEAM_LIST = ['SK']
for i in YEAR_LIST2:
    for j in TEAM_LIST:
        s1 = "%s_%d_ER = pitcher_%d.loc[pitcher_%d['T_ID'] == j]['GDAY_DS'].drop_duplicates().reset_index(drop = True)[120:]"%(j, i, i, i)
        exec(s1)
        
#키움 잔여경기 20경기        
TEAM_LIST = ['WO']
for i in YEAR_LIST2:
    for j in TEAM_LIST:
        s1 = "%s_%d_ER = pitcher_%d.loc[pitcher_%d['T_ID'] == j]['GDAY_DS'].drop_duplicates().reset_index(drop = True)[124:]"%(j, i, i, i)
        exec(s1)

#최종성적 ER예측템플릿
TEAM_LIST = ['LG', 'SS', 'HH', 'LT', 'NC', 'HT', 'WO', 'SK', 'KT', 'OB']
for i in YEAR_LIST2:
    for j in TEAM_LIST:
        s2 = "train_%s_%d_ER = pd.merge(%s_%d_ER, pitcher_%d.loc[pitcher_%d['T_ID'] == j], on = 'GDAY_DS', how = 'left')"%(j, i, j, i, i, i)
        exec(s2)
        
#Y변수 정제
for i in YEAR_LIST2:
    for j in TEAM_LIST:
        s1 = "train_%s_%d_ER = train_%s_%d_ER.iloc[:,:5].rename(columns = {'ER' : 'ER_REMAIN', 'INN2' : 'INN2_REMAIN'})"%(j,i,j,i)
        exec(s1)

In [112]:
#잔여경기를 제외한 누적기록
for i in TEAM_LIST:
    for j in YEAR_LIST2:
        s1 = "train_%s_%d = train_%s_%d.groupby(['NAME', 'T_ID'])['ER', 'INN2', 'BF', 'PA', 'AB', 'HIT', 'H2', 'H3', 'HR', 'BB', 'IB', 'HP', 'KK', 'R'].sum().reset_index()"%(i,j,i,j)
        exec(s1)
        
#잔여경기 누적기록
for i in TEAM_LIST:
    for j in YEAR_LIST2:
        s1 = "train_%s_%d_ER = train_%s_%d_ER.groupby(['NAME', 'T_ID'])['ER_REMAIN', 'INN2_REMAIN'].sum().reset_index()"%(i,j,i,j)
        exec(s1)

In [113]:
#연도 컬럼 추가
for i in TEAM_LIST:
    for j in YEAR_LIST2:
        if j == 2016:
            s1 = "train_%s_%d['GYEAR'] = 2016"%(i,j)
        elif j == 2017:
            s2 = "train_%s_%d['GYEAR'] = 2017"%(i,j)
        elif j == 2018:
            s3 = "train_%s_%d['GYEAR'] = 2018"%(i,j)
        elif j == 2019:
            s4 = "train_%s_%d['GYEAR'] = 2019"%(i,j)
        else:
            s5 = "train_%s_%d['GYEAR'] = 2020"%(i,j)
            exec(s1)
            exec(s2)
            exec(s3)
            exec(s4)
            exec(s5)

In [114]:
#독립, 종속변후 병합
for i in YEAR_LIST2:
    for j in TEAM_LIST:
        s1 = "train_%s_%d = pd.merge(train_%s_%d, train_%s_%d_ER, on = ['NAME', 'T_ID'], how = 'left')"%(j,i,j,i,j,i)
        exec(s1)

In [115]:
#시즌 누적 FIP 구하기
for i in TEAM_LIST:
    for j in YEAR_LIST2:
        if j == 2016:
            s1 = "train_%s_%d['FIP'] = ((-2*train_%s_%d['KK'] + 3*(train_%s_%d['BB'] + train_%s_%d['IB'] + train_%s_%d['HP']) + 13*(train_%s_%d['HR']))/(train_%s_%d['INN2']*(1/3))) + 3.792"%(i,j, i,j, i,j, i,j, i,j, i,j, i,j)
        elif j == 2017:
            s2 = "train_%s_%d['FIP'] = ((-2*train_%s_%d['KK'] + 3*(train_%s_%d['BB'] + train_%s_%d['IB'] + train_%s_%d['HP']) + 13*(train_%s_%d['HR']))/(train_%s_%d['INN2']*(1/3))) + 3.735"%(i,j, i,j, i,j, i,j, i,j, i,j, i,j)
        elif j == 2018:
            s3 = "train_%s_%d['FIP'] = ((-2*train_%s_%d['KK'] + 3*(train_%s_%d['BB'] + train_%s_%d['IB'] + train_%s_%d['HP']) + 13*(train_%s_%d['HR']))/(train_%s_%d['INN2']*(1/3))) + 3.833"%(i,j, i,j, i,j, i,j, i,j, i,j, i,j)
        elif j == 2019:
            s4 = "train_%s_%d['FIP'] = ((-2*train_%s_%d['KK'] + 3*(train_%s_%d['BB'] + train_%s_%d['IB'] + train_%s_%d['HP']) + 13*(train_%s_%d['HR']))/(train_%s_%d['INN2']*(1/3))) + 3.403"%(i,j, i,j, i,j, i,j, i,j, i,j, i,j)
        elif j == 2020:
            s5 = "train_%s_%d['FIP'] = ((-2*train_%s_%d['KK'] + 3*(train_%s_%d['BB'] + train_%s_%d['IB'] + train_%s_%d['HP']) + 13*(train_%s_%d['HR']))/(train_%s_%d['INN2']*(1/3))) + 3.609"%(i,j, i,j, i,j, i,j, i,j, i,j, i,j)
            
            exec(s1)
            exec(s2)
            exec(s3)
            exec(s4)
            exec(s5)

In [116]:
#연도 병합
YEAR_1 = [2016]
YEAR_2 = [2017]
YEAR_3 = [2018]
YEAR_4 = [2019]
for i in TEAM_LIST:
    for a, b, c, d in zip(YEAR_1, YEAR_2, YEAR_3, YEAR_4):
        s1 = "train_%s = pd.concat([train_%s_%d, train_%s_%d, train_%s_%d, train_%s_%d], axis = 0)"%(i, i,a, i,b, i,c, i,d)
        exec(s1)

In [117]:
#train = 2016 ~ 2019
train = pd.concat([train_HT, train_NC, train_LT, train_LG, train_SS,
                  train_HH, train_KT, train_OB, train_WO, train_SK], axis = 0)

#test = 2020
test = pd.concat([train_HT_2020, train_NC_2020, train_LT_2020, train_LG_2020, train_SS_2020,
                  train_HH_2020, train_KT_2020, train_OB_2020, train_WO_2020, train_SK_2020], axis = 0)

In [126]:
#최종성적 생성
train['FINAL_ER'] = train['ER'] + train['ER_REMAIN']
train['FINAL_INN2'] = train['INN2'] + train['INN2_REMAIN']
train['FINAL_ERA'] = (train['FINAL_ER']*27)/train['FINAL_INN2']

,NAME,T_ID,ER,INN2,BF,PA,AB,HIT,H2,H3,...,GYEAR,ER_REMAIN,INN2_REMAIN,FIP,ERA,WHIP,H1,FINAL_ER,FINAL_INN2,FINAL_ERA
0,고효준,HT,4,55,325,77,64,11,3,1,...,2016,13.0,54.0,4.555636,1.963636,1.309091,6,17.0,109.0,4.211009
1,곽정철,HT,21,78,479,125,103,31,8,1,...,2016,2.0,4.0,5.753538,7.269231,1.884615,20,23.0,82.0,7.573171
2,김광수,HT,24,146,854,215,194,57,8,1,...,2016,5.0,11.0,4.531726,4.438356,1.520548,44,29.0,157.0,4.987261
3,김명찬,HT,6,29,161,44,40,13,3,0,...,2016,1.0,10.0,4.516138,5.586207,1.655172,9,7.0,39.0,4.846154
4,김윤동,HT,25,91,623,149,118,33,7,0,...,2016,6.0,68.0,7.978813,7.417582,2.010989,19,31.0,159.0,5.264151
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
964,정영일,SK,8,89,485,120,101,19,1,0,...,2019,7.0,37.0,3.672663,2.426966,1.179775,18,15.0,126.0,3.214286
965,정재원,SK,5,8,50,16,15,6,0,0,...,2019,0.0,0.0,8.653000,16.875000,2.250000,5,5.0,8.0,16.875000
966,조영우,SK,11,37,224,57,51,19,2,0,...,2019,0.0,0.0,6.240838,8.027027,1.864865,14,11.0,37.0,8.027027
967,채병용,SK,4,11,84,21,16,7,2,0,...,2019,0.0,0.0,8.584818,9.818182,2.727273,4,4.0,11.0,9.818182


In [118]:
#FIP NAN값 제거
drop_list = test.loc[test['FIP'].isnull()].index
test = test.drop(drop_list).reset_index()

#무한값 제거
drop_list = test.loc[test['FIP'] > 10000].index
test = test.drop(drop_list)
test = test.reset_index(drop=True)

In [119]:
#train, test Nan제거
train = train.fillna(0).reset_index(drop=True)
test = test.dropna(axis = 1).reset_index(drop=True)
test = test.drop(['index'], axis = 1)

In [120]:
#ERA 생성
train['ERA'] = (train['ER']*27)/train['INN2']
test['ERA'] = (test['ER']*27)/test['INN2']
#WHIP 생성
train['WHIP'] = (train['HIT']+train['BB'])/(train['INN2']*(1/3))
test['WHIP'] = (test['HIT']+test['BB'])/(test['INN2']*(1/3))
#H1 생성
train['H1'] = train['HIT'] - train['HR'] - train['H3'] - train['H2']
test['H1'] = test['HIT'] - test['HR'] - test['H3'] - test['H2']

In [70]:
# indicator variable (잔여경기 출전선수)

a=[]
for k in range(train.shape[0]):
    if train['INN2_REMAIN'][k]==0 :
        a.append(0)
    else :
        a.append(1)
        
train['Not_zero']=a

In [71]:
#잔여경기 변수 추가(train)
train['remain'] = ''
for i in range(train.shape[0]):
    if train['T_ID'][i] == 'HT':
        train['remain'][i] = 29
    elif train['T_ID'][i] == 'NC' or train['T_ID'][i] == 'LT':
        train['remain'][i] = 28
    elif train['T_ID'][i] == 'KT':
        train['remain'][i] = 27
    elif train['T_ID'][i] == 'HH' or train['T_ID'][i] == 'SS':
        train['remain'][i] = 26
    elif train['T_ID'][i] == 'OB' or train['T_ID'][i] == 'LG':
        train['remain'][i] = 25
    elif train['T_ID'][i] == 'SK':
        train['remain'][i] = 24
    else:
        train['remain'][i] = 20
        
#잔여경기 변수 추가(test)
test['remain'] = ''
for i in range(test.shape[0]):
    if test['T_ID'][i] == 'HT':
        test['remain'][i] = 29
    elif test['T_ID'][i] == 'NC' or test['T_ID'][i] == 'LT':
        test['remain'][i] = 28
    elif test['T_ID'][i] == 'KT':
        test['remain'][i] = 27
    elif test['T_ID'][i] == 'HH' or test['T_ID'][i] == 'SS':
        test['remain'][i] = 26
    elif test['T_ID'][i] == 'OB' or test['T_ID'][i] == 'LG':
        test['remain'][i] = 25
    elif test['T_ID'][i] == 'SK':
        test['remain'][i] = 24
    else:
        test['remain'][i] = 20

C:\Users\Kim\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\Kim\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\Kim\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\Users\Kim\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: Setting

In [72]:
#출전여부를 위한 데이터
zero_train=train[['GYEAR', 'NAME', 'T_ID', 'ERA', 'HIT', 'H1', 'H2', 'H3', 'HR', 'BB','IB', 'HP', 'KK', 'INN2',
                  'ER', 'BF', 'PA', 'AB', 'WHIP', 'FIP', 'Not_zero', 'INN2_REMAIN', 'ER_REMAIN', 'remain']]

In [73]:
#train, test 나누기
train_x=train.drop(['GYEAR', 'NAME','T_ID', 'ER_REMAIN', 'INN2_REMAIN', 'Not_zero'], axis=1)
train_y=train['Not_zero']


test_x=test.drop(['GYEAR', 'NAME','T_ID'], axis=1)

In [74]:
#출전선수 예측을 위한 분류 모델
lglasso=LogisticRegression(penalty='l1', solver='liblinear', C=1)
lglasso_params={'C':[0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 0, 1, 2, 3]}
lglasso_cv=GridSearchCV(lglasso, lglasso_params, cv=5)
lglasso_cv.fit(train_x, train_y)
print(lglasso_cv.best_params_)
pred = lglasso_cv.predict(test_x)

C:\Users\Kim\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Kim\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Kim\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1360, in fit
    sample_weight=sample_weight)
  File "C:\Users\Kim\Anaconda3\lib\site-packages\sklearn\svm\_base.py", line 969, in _fit_liblinear
    epsilon, sample_weight)
  File "sklearn\svm\_liblinear.pyx", line 52, in sklearn.svm._liblinear.train_wrap
ValueError: b'C <= 0'

  FitFailedWarning)
C:\Users\Kim\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be

{'C': 0.01}


In [75]:
#출전여부 컴럼 생성
test['pred_zero'] = pred

In [76]:
#출전선수로만 이루어진 데이터
train = train.loc[train['Not_zero'] == 1].reset_index(drop=True)
test = test.loc[test['pred_zero'] == 1].reset_index(drop=True)

In [77]:
#최종 예측 템플릿
x_train=train[['HIT', 'H1', 'H2', 'H3', 'HR', 'BB', 'KK', 'IB', 'HP', 'INN2', 'ER', 'BF', 'PA', 'AB']]
y_train = train['ER_REMAIN']
x_test = test[['HIT', 'H1', 'H2', 'H3', 'HR', 'BB', 'KK', 'IB', 'HP', 'INN2', 'ER', 'BF', 'PA', 'AB']]

In [78]:
#ER예측을 위한 회귀모델
lasso_er=Lasso()
lasso_params={'alpha':[0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 0, 1, 2, 3]}
lasso_cv=GridSearchCV(lasso_er, lasso_params, cv=5)
lasso_cv.fit(x_train,y_train)
print(lasso_cv.best_params_)

pred_er=lasso_cv.predict(x_test)

C:\Users\Kim\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5552.17838783574, tolerance: 1.716704406130268
  positive)
C:\Users\Kim\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5758.335092489623, tolerance: 1.725441379310345
  positive)
C:\Users\Kim\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5164.99912431248, tolerance: 1.717506704980843
  positive)
C:\Users\Kim\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5732.1

C:\Users\Kim\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2493.097697101927, tolerance: 1.8688271510516254
  positive)
C:\Users\Kim\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:531: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Kim\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  positive)
C:\Users\Kim\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5554.505089645962, tolerance: 1.716704406130268
  positive)
C:\Users\K

{'alpha': 0}


C:\Users\Kim\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 476.14249216786266, tolerance: 1.8688271510516254
  positive)
C:\Users\Kim\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 50.532431663321404, tolerance: 1.716704406130268
  positive)
C:\Users\Kim\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6.344034751107756, tolerance: 1.725441379310345
  positive)
C:\Users\Kim\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1

In [79]:
#예측값
test['ER_REMAIN']=pred_er

In [80]:
#ERA예측값
team_result=test.groupby('T_ID').sum()
#잔여경기수 * 27
team_result['INN2_REMAIN'] = [702, 783, 729, 675, 756, 756, 675, 648, 702, 540]
team_ERA=team_result['ER_REMAIN']*27/team_result['INN2_REMAIN']
team_ERA

T_ID
HH    4.782181
HT    3.676207
KT    4.011476
LG    4.176391
LT    4.374373
NC    4.914364
OB    5.180596
SK    5.671931
SS    4.833324
WO    6.125032
dtype: float64

# 타자 모델링(재현)

In [82]:
#데이터 불러오기
YEAR_LIST = [2016, 2017, 2018, 2019]
for i in YEAR_LIST:
    s1 = "batter_%d = pd.read_csv(r'C:\\Users\\Kim\\Desktop\\빅콘테스트\\데이터\\기록\\개인타자\\2020빅콘테스트_스포츠투아이_제공데이터_개인타자_%d.csv', encoding='cp949')"%(i, i)
    exec(s1)
    
for i in YEAR_LIST2:
    s2 = "player_%d = pd.read_csv(r'C:\\Users\\Kim\\Desktop\\빅콘테스트\\데이터\\기록\\선수\\2020빅콘테스트_스포츠투아이_제공데이터_선수_%d.csv', encoding='cp949')"%(i, i)
    exec(s2)

#크롤링 데이터    
batter_2020 = pd.read_csv('2020최신타자(선수별).csv')

In [83]:
#선수이름 데이터 생성
for i in YEAR_LIST2:
    s1 = "player_%d = pd.concat([player_%d['PCODE'], player_%d['NAME'], player_%d['AGE_VA']], axis = 1).rename(columns = {'PCODE' : 'P_ID'})"%(i, i, i, i)
    exec(s1)
    
#기록데이터, 선수이름 데이터 병합    
for i in YEAR_LIST:
    s1 = "batter_%d = pd.merge(batter_%d, player_%d, on ='P_ID', how = 'left')"%(i, i, i)
    exec(s1)


#사용하는 변수만 연도별로 뽑아놓기
YEAR_LIST2 = [2016, 2017, 2018, 2019, 2020]
for i in YEAR_LIST2:
    s1 = "batter_%d = batter_%d[['GDAY_DS', 'NAME', 'T_ID', 'AB', 'HIT', 'RBI', 'RUN', 'BB', 'HP', 'SB', 'CS', 'GD', 'H2', 'H3', 'HR', 'KK']]"%(i,i)
    exec(s1)

In [84]:
#잔여경기 일정에 맞게 날짜 데이터 추출

#기어 잔여경기 29경기
TEAM_LIST = ['HT']
for i in YEAR_LIST2:
    for j in TEAM_LIST:
        s1 = "%s_%d = batter_%d.loc[batter_%d['T_ID'] == j]['GDAY_DS'].drop_duplicates().reset_index(drop = True)[:115]"%(j, i, i, i)
        exec(s1)
        
#NC, 롯데 잔여경기 28경기
TEAM_LIST = ['NC', 'LT']
for i in YEAR_LIST2:
    for j in TEAM_LIST:
        s1 = "%s_%d = batter_%d.loc[batter_%d['T_ID'] == j]['GDAY_DS'].drop_duplicates().reset_index(drop = True)[:116]"%(j, i, i, i)
        exec(s1)
        
#KT 잔여경기 27경기        
TEAM_LIST = ['KT']
for i in YEAR_LIST2:
    for j in TEAM_LIST:
        s1 = "%s_%d = batter_%d.loc[batter_%d['T_ID'] == j]['GDAY_DS'].drop_duplicates().reset_index(drop = True)[:117]"%(j, i, i, i)
        exec(s1)
        
#한화, 삼성 잔여경기 26경기        
TEAM_LIST = ['HH', 'SS']
for i in YEAR_LIST2:
    for j in TEAM_LIST:
        s1 = "%s_%d = batter_%d.loc[batter_%d['T_ID'] == j]['GDAY_DS'].drop_duplicates().reset_index(drop = True)[:118]"%(j, i, i, i)
        exec(s1)
        
#두산, LG 잔여경기 25경기        
TEAM_LIST = ['OB', 'LG']
for i in YEAR_LIST2:
    for j in TEAM_LIST:
        s1 = "%s_%d = batter_%d.loc[batter_%d['T_ID'] == j]['GDAY_DS'].drop_duplicates().reset_index(drop = True)[:119]"%(j, i, i, i)
        exec(s1)
        
#SK 잔여경기 24경기        
TEAM_LIST = ['SK']
for i in YEAR_LIST2:
    for j in TEAM_LIST:
        s1 = "%s_%d = batter_%d.loc[batter_%d['T_ID'] == j]['GDAY_DS'].drop_duplicates().reset_index(drop = True)[:120]"%(j, i, i, i)
        exec(s1)
        
#키움 잔여경기 20경기        
TEAM_LIST = ['WO']
for i in YEAR_LIST2:
    for j in TEAM_LIST:
        s1 = "%s_%d = batter_%d.loc[batter_%d['T_ID'] == j]['GDAY_DS'].drop_duplicates().reset_index(drop = True)[:124]"%(j, i, i, i)
        exec(s1)

In [85]:
#잔여경기를 제외한 경기 기록(Train)
TEAM_LIST = ['LG', 'SS', 'HH', 'LT', 'NC', 'HT', 'WO', 'SK', 'KT', 'OB']
for i in YEAR_LIST2:
    for j in TEAM_LIST:
        s2 = "train_%s_%d = pd.merge(%s_%d, batter_%d.loc[batter_%d['T_ID'] == j], on = 'GDAY_DS', how = 'left')"%(j, i, j, i, i, i)
        exec(s2)

In [86]:
#잔여경기를 제외한 경기 기록(Train)
TEAM_LIST = ['LG', 'SS', 'HH', 'LT', 'NC', 'HT', 'WO', 'SK', 'KT', 'OB']
for i in YEAR_LIST2:
    for j in TEAM_LIST:
        s2 = "train_%s_%d = pd.merge(%s_%d, batter_%d.loc[batter_%d['T_ID'] == j], on = 'GDAY_DS', how = 'left')"%(j, i, j, i, i, i)
        exec(s2)
        
#기어 잔여경기 29경기
TEAM_LIST = ['HT']
for i in YEAR_LIST2:
    for j in TEAM_LIST:
        s1 = "%s_%d_HIT = batter_%d.loc[batter_%d['T_ID'] == j]['GDAY_DS'].drop_duplicates().reset_index(drop = True)[115:]"%(j, i, i, i)
        exec(s1)
        
#NC, 롯데 잔여경기 28경기
TEAM_LIST = ['NC', 'LT']
for i in YEAR_LIST2:
    for j in TEAM_LIST:
        s1 = "%s_%d_HIT = batter_%d.loc[batter_%d['T_ID'] == j]['GDAY_DS'].drop_duplicates().reset_index(drop = True)[116:]"%(j, i, i, i)
        exec(s1)
        
#KT 잔여경기 27경기        
TEAM_LIST = ['KT']
for i in YEAR_LIST2:
    for j in TEAM_LIST:
        s1 = "%s_%d_HIT = batter_%d.loc[batter_%d['T_ID'] == j]['GDAY_DS'].drop_duplicates().reset_index(drop = True)[117:]"%(j, i, i, i)
        exec(s1)
        
#한화, 삼성 잔여경기 26경기        
TEAM_LIST = ['HH', 'SS']
for i in YEAR_LIST2:
    for j in TEAM_LIST:
        s1 = "%s_%d_HIT = batter_%d.loc[batter_%d['T_ID'] == j]['GDAY_DS'].drop_duplicates().reset_index(drop = True)[118:]"%(j, i, i, i)
        exec(s1)
        
#두산, LG 잔여경기 25경기        
TEAM_LIST = ['OB', 'LG']
for i in YEAR_LIST2:
    for j in TEAM_LIST:
        s1 = "%s_%d_HIT = batter_%d.loc[batter_%d['T_ID'] == j]['GDAY_DS'].drop_duplicates().reset_index(drop = True)[119:]"%(j, i, i, i)
        exec(s1)
        
#SK 잔여경기 24경기        
TEAM_LIST = ['SK']
for i in YEAR_LIST2:
    for j in TEAM_LIST:
        s1 = "%s_%d_HIT = batter_%d.loc[batter_%d['T_ID'] == j]['GDAY_DS'].drop_duplicates().reset_index(drop = True)[120:]"%(j, i, i, i)
        exec(s1)
        
#키움 잔여경기 20경기        
TEAM_LIST = ['WO']
for i in YEAR_LIST2:
    for j in TEAM_LIST:
        s1 = "%s_%d_HIT = batter_%d.loc[batter_%d['T_ID'] == j]['GDAY_DS'].drop_duplicates().reset_index(drop = True)[124:]"%(j, i, i, i)
        exec(s1)

#최종성적 ER예측템플릿
TEAM_LIST = ['LG', 'SS', 'HH', 'LT', 'NC', 'HT', 'WO', 'SK', 'KT', 'OB']
for i in YEAR_LIST2:
    for j in TEAM_LIST:
        s2 = "train_%s_%d_HIT = pd.merge(%s_%d_HIT, batter_%d.loc[batter_%d['T_ID'] == j], on = 'GDAY_DS', how = 'left')"%(j, i, j, i, i, i)
        exec(s2)
        
#Y변수 정제
for i in YEAR_LIST2:
    for j in TEAM_LIST:
        s1 = "train_%s_%d_HIT = train_%s_%d_HIT.iloc[:,:5].rename(columns = {'AB' : 'AB_REMAIN', 'HIT' : 'HIT_REMAIN'})"%(j,i,j,i)
        exec(s1)

In [87]:
#잔여경기를 제외한 누적기록
for i in TEAM_LIST:
    for j in YEAR_LIST2:
        s1 = "train_%s_%d = train_%s_%d.groupby(['NAME', 'T_ID'])['AB', 'HIT', 'RBI', 'RUN', 'BB', 'HP', 'SB', 'CS', 'GD', 'H2', 'H3', 'HR', 'KK'].sum().reset_index()"%(i,j,i,j)
        exec(s1)
        
#잔여경기 누적기록
for i in TEAM_LIST:
    for j in YEAR_LIST2:
        s1 = "train_%s_%d_HIT = train_%s_%d_HIT.groupby(['NAME', 'T_ID'])['AB_REMAIN', 'HIT_REMAIN'].sum().reset_index()"%(i,j,i,j)
        exec(s1)

In [88]:
#연도 컬럼 추가
for i in TEAM_LIST:
    for j in YEAR_LIST2:
        if j == 2016:
            s1 = "train_%s_%d['GYEAR'] = 2016"%(i,j)
        elif j == 2017:
            s2 = "train_%s_%d['GYEAR'] = 2017"%(i,j)
        elif j == 2018:
            s3 = "train_%s_%d['GYEAR'] = 2018"%(i,j)
        elif j == 2019:
            s4 = "train_%s_%d['GYEAR'] = 2019"%(i,j)
        else:
            s5 = "train_%s_%d['GYEAR'] = 2020"%(i,j)
            exec(s1)
            exec(s2)
            exec(s3)
            exec(s4)
            exec(s5)

In [89]:
#독립, 종속변후 병합
for i in YEAR_LIST2:
    for j in TEAM_LIST:
        s1 = "train_%s_%d = pd.merge(train_%s_%d, train_%s_%d_HIT, on = ['NAME', 'T_ID'], how = 'left')"%(j,i,j,i,j,i)
        exec(s1)

In [90]:
#연도 병합
YEAR_1 = [2016]
YEAR_2 = [2017]
YEAR_3 = [2018]
YEAR_4 = [2019]
for i in TEAM_LIST:
    for a, b, c, d in zip(YEAR_1, YEAR_2, YEAR_3, YEAR_4):
        s1 = "train_%s = pd.concat([train_%s_%d, train_%s_%d, train_%s_%d, train_%s_%d], axis = 0)"%(i, i,a, i,b, i,c, i,d)
        exec(s1)

In [91]:
train = pd.concat([train_HT, train_NC, train_LT, train_LG, train_SS,
                  train_HH, train_KT, train_OB, train_WO, train_SK], axis = 0)

test = pd.concat([train_HT_2020, train_NC_2020, train_LT_2020, train_LG_2020, train_SS_2020,
                  train_HH_2020, train_KT_2020, train_OB_2020, train_WO_2020, train_SK_2020], axis = 0)

In [92]:
#Nan값처리
train['AB_REMAIN'] = train['AB_REMAIN'].fillna(0)
train['HIT_REMAIN'] = train['HIT_REMAIN'].fillna(0)

In [93]:
#1루타 생성
train['H1'] = train['HIT'] - train['HR'] - train['H3'] - train['H2']
test['H1'] = test['HIT'] - test['HR'] - test['H3'] - test['H2']
#장타율 생성
train['PER_LONG'] = ((train['H1'] + (2*train['H2']) + (3*train['H3']) + (4*train['HR']))/train['AB'])
test['PER_LONG'] = ((test['H1'] + (2*test['H2']) + (3*test['H3']) + (4*test['HR']))/test['AB'])
#타율 생성
train['PER_HIT'] = train['HIT']/train['AB']
test['PER_HIT'] = test['HIT']/test['AB']

In [94]:
#장타율, 타율 nan값처리
train['PER_LONG'] = train['PER_LONG'].fillna(0)
train['PER_HIT'] = train['PER_HIT'].fillna(0)
test['PER_LONG'] = test['PER_LONG'].fillna(0)
test['PER_HIT'] = test['PER_HIT'].fillna(0)

In [95]:
#출전선수 구분을 위한 변수
train['Not_zero'] = train['AB_REMAIN']>0
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)

In [96]:
#잔여경기 변수 추가(train)
train['remain'] = ''
for i in range(train.shape[0]):
    if train['T_ID'][i] == 'HT':
        train['remain'][i] = 29
    elif train['T_ID'][i] == 'NC' or train['T_ID'][i] == 'LT':
        train['remain'][i] = 28
    elif train['T_ID'][i] == 'KT':
        train['remain'][i] = 27
    elif train['T_ID'][i] == 'HH' or train['T_ID'][i] == 'SS':
        train['remain'][i] = 26
    elif train['T_ID'][i] == 'OB' or train['T_ID'][i] == 'LG':
        train['remain'][i] = 25
    elif train['T_ID'][i] == 'SK':
        train['remain'][i] = 24
    else:
        train['remain'][i] = 20
        
#잔여경기 변수 추가(test)
test['remain'] = ''
for i in range(test.shape[0]):
    if test['T_ID'][i] == 'HT':
        test['remain'][i] = 29
    elif test['T_ID'][i] == 'NC' or test['T_ID'][i] == 'LT':
        test['remain'][i] = 28
    elif test['T_ID'][i] == 'KT':
        test['remain'][i] = 27
    elif test['T_ID'][i] == 'HH' or test['T_ID'][i] == 'SS':
        test['remain'][i] = 26
    elif test['T_ID'][i] == 'OB' or test['T_ID'][i] == 'LG':
        test['remain'][i] = 25
    elif test['T_ID'][i] == 'SK':
        test['remain'][i] = 24
    else:
        test['remain'][i] = 20

C:\Users\Kim\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\Kim\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\Kim\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\Users\Kim\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: Setting

In [97]:
#train, test 나누기
train_x=train.drop(['GYEAR', 'NAME','T_ID', 'AB_REMAIN', 'HIT_REMAIN', 'Not_zero'], axis=1)
train_y=train['Not_zero']


test_x=test.drop(['GYEAR', 'NAME','T_ID', 'AB_REMAIN', 'HIT_REMAIN'], axis=1)

In [98]:
lglasso=LogisticRegression(penalty='l1', solver='liblinear')
lglasso_params={'C' : [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 0, 1, 2, 3, 5, 10]}
lglasso_cv=GridSearchCV(lglasso, lglasso_params, cv=5)
lglasso_cv.fit(train_x, train_y)

C:\Users\Kim\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Kim\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Kim\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1360, in fit
    sample_weight=sample_weight)
  File "C:\Users\Kim\Anaconda3\lib\site-packages\sklearn\svm\_base.py", line 969, in _fit_liblinear
    epsilon, sample_weight)
  File "sklearn\svm\_liblinear.pyx", line 52, in sklearn.svm._liblinear.train_wrap
ValueError: b'C <= 0'

  FitFailedWarning)
C:\Users\Kim\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be

GridSearchCV(cv=5,
             estimator=LogisticRegression(penalty='l1', solver='liblinear'),
             param_grid={'C': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1,
                               0.5, 0, 1, 2, 3, 5, 10]})

In [99]:
#출전여부 컴럼 생성
pred = lglasso_cv.predict(test_x)
test['pred_zero'] = pred

In [100]:
#안타개수 예측 데이터
x_train = train[train['Not_zero']==True].reset_index(drop=True).drop(['NAME', 'T_ID', 'AB_REMAIN', 'HIT_REMAIN', 'Not_zero'], axis = 1)
y_train = train[train['Not_zero']==True].reset_index(drop=True)['HIT_REMAIN']
x_test = test[test['pred_zero']==True].reset_index(drop=True).drop(['NAME', 'T_ID', 'AB_REMAIN', 'HIT_REMAIN', 'pred_zero'], axis = 1)

In [101]:
lasso_HIT=Lasso()
lasso_params={'alpha':[0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 0, 1, 2, 3]}
lasso_cv=GridSearchCV(lasso_HIT, lasso_params, cv=5)
lasso_cv.fit(x_train, y_train)
lasso_cv.best_params_

C:\Users\Kim\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5040.619763519084, tolerance: 7.460860606060606
  positive)
C:\Users\Kim\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2675.133589485322, tolerance: 7.659896012759171
  positive)
C:\Users\Kim\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5165.476219532227, tolerance: 7.811292185007973
  positive)
C:\Users\Kim\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5096

C:\Users\Kim\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:531: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Kim\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  positive)
C:\Users\Kim\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 15886.794421719022, tolerance: 7.460860606060606
  positive)
C:\Users\Kim\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:531: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Kim\Anaconda

{'alpha': 0.005}

In [102]:
#예측
pred_HIT=lasso_cv.predict(x_test)

In [103]:
#출전 선수만 인덱싱한 후 예측결과 반영
pred_temp = test.loc[test['pred_zero'] == True].reset_index(drop=True)
pred_temp['PRED_HIT'] = pred_HIT
pred_temp = pred_temp.groupby(['T_ID']).sum()
#잔여경기수 * 27
pred_temp['PRED_AB'] = [702, 783, 729, 648, 756, 756, 675, 648, 702, 540]

In [104]:
#타자 예측값
pred_temp['PRED_HIT']/pred_temp['PRED_AB']

T_ID
HH    0.244786
HT    0.346115
KT    0.289816
LG    0.337916
LT    0.309652
NC    0.346387
OB    0.348291
SK    0.221800
SS    0.312219
WO    0.298801
dtype: float64